In [ ]:
# loading the dataset 
# loading from google drive

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# read the data using pandas
import pandas as pd

data = pd.read_csv("/content/drive/NYC/middleSchoolData.csv")
data.head()

In [ ]:
# trying another method to load the data from google drive
from google.colab import files 
uploaded = files.upload()

In [ ]:
import io
import pandas as pd
# include path to dataset if not using google colab
# data = pd.read_csv('/path/....')
data = pd.read_csv('middleSchoolData.csv')
data.head()

In [ ]:
data.info()

In [ ]:
# dataset got alot of missing values that need to be taken care off
data.describe()

In [ ]:
# plottng the data using a histogram to show the number of instances on the y and x gven range
import matplotlib.pyplot as plt
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Question 1
# correlation between the number of applications and admissions to HSPHS?
# splitting the data to fit to the models later
X = data["applications"].values.reshape(-1, 1)
Y = data["acceptances"]

In [ ]:
# visualizing applications against acceptance
# plot
plt.plot(data["applications"], data["acceptances"], 'o',
         markersize=2)
plt.xlabel('applications')
plt.ylabel('acceptances')

In [ ]:
# adding regression line to visualization for better corr
# regression model from sklearn 

from sklearn.linear_model import LinearRegression
import numpy as np

regres = LinearRegression()
regres.fit(X, Y)
reg_sqr = regres.score(X, Y)
r = np.sqrt(reg_sqr)
betas = regres.coef_  
y_int = regres.intercept_ 

y_hat = betas[0] * data["applications"] + \
    y_int 
# slope-intercept form, y = mx + b
plt.plot(data["applications"], y_hat, color='orange', linewidth=0.5,)
# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(reg_sqr, r))

In [ ]:
# r=0.802
# using pandas scatter_matrix function to check for correlation btn attributes
from pandas.plotting import scatter_matrix

attributes = [ "applications","acceptances"]
scatter_matrix(data[attributes], figsize=(12,8))

From the above out we can see that the correlation between applications and acceptances is greater.

In [ ]:
# Question 2
# better predictor of admission to HSPHS
# lets have a look at the atrributes

data.info()

In [ ]:
# taking care of missing values using the imputer class
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_data = pd.DataFrame(my_imputer.fit_transform(data))

# Imputation removed column names; put them back
imputed_data.columns = data.columns

data.info()

In [ ]:
# impute strategy wont work with categorical variables 
# trying dropna
# drop row with missing school size in tmp
tmp = data[["applications", "school_size", "acceptances"]]
tmp = tmp.dropna()

X = (tmp["applications"] /
     tmp["school_size"]).values.reshape(-1, 1)
Y = tmp["acceptances"]

# Plot application rates against acceptances
plt.plot(tmp["applications"] /
         tmp["school_size"], tmp["acceptances"], 'o',
         markersize=2)  
plt.xlabel('application rates')
plt.ylabel('acceptances')

#Add regression line to visualization
# slope-intercept form, y = mx + b
y_hat = betas[0] * tmp["applications"] / tmp["school_size"] + \
    y_int 
plt.plot(tmp["applications"] / tmp["school_size"],
         y_hat, color='orange', linewidth=0.5)
# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(reg_sqr, r))  # r = 0.802

In [ ]:
# using the correlation it turns out that number of applications
# is a better predictor as compared to acceptance rate


In [ ]:
# Question 3
# Which school has the best *per student* odds of sending someone to HSPHS?

# drop row with missing school size in tmp
tmp = data[["school_name", "school_size", "acceptances"]]
tmp = tmp.dropna()

# greater probability always result in greater odd, so we only need to find out
# schools with the highest proportion of student being admitted to HSPHS

tmp.loc[(tmp["acceptances"] / tmp["school_size"]).idxmax()]

From the above output, it is evident that CHRISTA MCAULIFFE SCHOOL\I.S has the best odds per student of sending someone to HSPHS.

In [ ]:
# Question 4
# relationship between how students perceive their school (as reported in columns
# L-Q) and how the school performs on objective measures of achievement (as noted in
# columns V-X)

# "school climate factors", drop row with missing data
climate = data.iloc[:, 11:17]
climate = climate.dropna()

# Z-score the data:
from scipy import stats
zscored_cilmate = stats.zscore(climate)

In [ ]:
# PCA to reduce number of dimension to 1
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(zscored_cilmate)

eig_vals = pca.explained_variance_
loadings = pca.components_
rotated_data = pca.fit_transform(zscored_cilmate)

In [ ]:
# plot of a bar graph of eigenvalues
plt.bar(np.linspace(1, 6, 6), eig_vals)
plt.xlabel('Principal component')
plt.ylabel('Eigenvalue')
plt.plot([1, 6], [1, 1], color='red',
         linewidth=1)  # Kaiser criterion line


In [ ]:
# add back to the temporary dataframe first to restore the index in the original
# dataframe (we drop some data but want to keep orginal index)

# sometime PCA will flip the sign, we need to flip it back to make this rating
# intuitive (higher is better)
climate["climate_rating"] = -rotated_data[:, 0]

# insert to original data frame
data["climate_rating"] = climate["climate_rating"]

# "measures of achievement", drop row with missing data
achievement = data.iloc[:, 21:24]
achievement = achievement.dropna()

# Z-score the data:
zscored_achievement = stats.zscore(achievement)

# do a PCA to reduce number of dimension to 1
pca = PCA()
pca.fit(zscored_achievement)

eig_vals = pca.explained_variance_
loadings = pca.components_
rotated_data = pca.fit_transform(zscored_achievement)

In [ ]:
# What a scree plot is: Plotting a bar graph of the sorted Eigenvalues
plt.figure()
plt.bar(np.linspace(1, 3, 3), eig_vals)
plt.xlabel('Principal component')
plt.ylabel('Eigenvalue')
plt.plot([1, 3], [1, 1], color='red',
         linewidth=1)  # Kaiser criterion line

In [ ]:
# add back to the temporary dataframe first to restore the index in the original
# dataframe (we drop some data but want to keep orginal index)
achievement["achievement_rating"] = rotated_data[:, 0]

# insert to original data frame
data["achievement_rating"] = achievement["achievement_rating"]

# finally, do a simple regression
tmp = data[["climate_rating", "achievement_rating"]]
tmp = tmp.dropna()

X = tmp["climate_rating"].values.reshape(-1, 1)
Y = tmp["achievement_rating"]


plt.figure()
plt.plot(tmp["climate_rating"], tmp["achievement_rating"], 'o',
         markersize=2)  # Plot application rates against acceptances
plt.xlabel('climate ratings')
plt.ylabel('achievement ratings')

In [ ]:
# Adding regression line to visualization:
y_hat = betas[0] * tmp["climate_rating"] + \
    y_int
# slope-intercept form, y = mx + b
plt.plot(tmp["climate_rating"], y_hat, color='orange', linewidth=0.5)

# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(reg_sqr, r))

It is evident that their is a relationship between how students perceive their school and how the school performs. 

In [ ]:
# Question 5
# which kind of school
# performs differently than another kind either on some dependent measure, e.g. objective
# measures of achievement or admission to HSPHS.

# using acceptance rate instead of raw acceptance numbers
data["acceptance_rate"] = data["acceptances"] / \
    data["applications"]

# creating a binary indicator for poor and rich schools
median_spending = data["per_pupil_spending"].median()
data["rich_school"] = data["per_pupil_spending"] > median_spending

# drop row with missing data in tmp
tmp = data[["rich_school", "achievement_rating"]]
tmp = tmp.dropna()

rich = tmp[tmp["rich_school"] == 1]["achievement_rating"]
poor = tmp[tmp["rich_school"] == 0]["achievement_rating"]
u_rich_achievement, p_rich_achievement = stats.mannwhitneyu(rich, poor)
p_rich_achievement


In [ ]:
# dropping row with missing data in tmp
tmp = data[["rich_school", "acceptance_rate"]]
tmp = tmp.dropna()

rich = tmp[tmp["rich_school"] == 1]["acceptance_rate"]
poor = tmp[tmp["rich_school"] == 0]["acceptance_rate"]
u_rich_acceptance, p_rich_acceptance = stats.mannwhitneyu(rich, poor)
p_rich_acceptance

Conducted mann-whittney u test on the performance of rich and poor schools on achievement rating and accpetance rates, which resulted in exteremely small p-values. 
This shows that poor schools and rich schools are extremely likely to perform differently when it comes to objective measures of achievements and admission to HSPHS

In [ ]:
# Question 6 
# evidence that the availability of material resources (e.g. per student spending
# or class size) impacts objective measures of achievement or admission to HSPHS?

# per pupil spending against achievement ratings
# drop row with missing data in tmp
tmp = data[["per_pupil_spending", "achievement_rating"]]
tmp = tmp.dropna()

X = tmp["per_pupil_spending"].values.reshape(-1, 1)
Y = tmp["achievement_rating"]

regr = LinearRegression().fit(X, Y)
r_sqr = regr.score(X, Y)
r = np.sqrt(r_sqr)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

plt.figure()
plt.plot(tmp["per_pupil_spending"], tmp["achievement_rating"], 'o',
         markersize=2)  # Plot per pupil spending against achievement ratings
plt.xlabel('spending per pupil')
plt.ylabel('achievement ratings')

In [ ]:
# Add regression line to visualization:
y_hat = betas[0] * tmp["per_pupil_spending"] + \
    y_int  # slope-intercept form, y = mx + b
plt.plot(tmp["per_pupil_spending"], y_hat, color='orange', linewidth=0.5)

# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(r_sqr, r))

In [ ]:
# per pupil spending against acceptance rate
# drop row with missing data in tmp
tmp = data[["per_pupil_spending", "acceptance_rate"]]
tmp = tmp.dropna()

X = tmp["per_pupil_spending"].values.reshape(-1, 1)
Y = tmp["acceptance_rate"]

regr = LinearRegression().fit(X, Y)
r_sqr = regr.score(X, Y)
r = np.sqrt(r_sqr)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

plt.figure()
plt.plot(tmp["per_pupil_spending"], tmp["acceptance_rate"], 'o',
         markersize=2)  # Plot per pupil spending against acceptance rate
plt.xlabel('spending per pupil')
plt.ylabel('acceptance rate')

In [ ]:
# Add regression line to visualization:
y_hat = betas[0] * tmp["per_pupil_spending"] + \
    y_int  # slope-intercept form, y = mx + b
plt.plot(tmp["per_pupil_spending"], y_hat, color='orange', linewidth=0.5)

# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(r_sqr, r))

In [ ]:
# average class sizes against achievement ratings
# drop row with missing data in tmp
tmp = data[["avg_class_size", "achievement_rating"]]
tmp = tmp.dropna()

X = tmp["avg_class_size"].values.reshape(-1, 1)
Y = tmp["achievement_rating"]

regr = LinearRegression().fit(X, Y)
r_sqr = regr.score(X, Y)
r = np.sqrt(r_sqr)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

plt.figure()
plt.plot(tmp["avg_class_size"], tmp["achievement_rating"], 'o',
         markersize=2)  # Plot average class sizes against achievement ratings
plt.xlabel('average class size')
plt.ylabel('achievement ratings')

In [ ]:
# Add regression line to visualization:
y_hat = betas[0] * tmp["avg_class_size"] + \
    y_int  # slope-intercept form, y = mx + b
plt.plot(tmp["avg_class_size"], y_hat, color='orange', linewidth=0.5)

# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(r_sqr, r))

In [ ]:
# average class sizes against acceptance rate
# drop row with missing data in tmp
tmp = data[["avg_class_size", "acceptance_rate"]]
tmp = tmp.dropna()

X = tmp["avg_class_size"].values.reshape(-1, 1)
Y = tmp["acceptance_rate"]

regr = LinearRegression().fit(X, Y)
r_sqr = regr.score(X, Y)
r = np.sqrt(r_sqr)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

plt.figure()
plt.plot(tmp["avg_class_size"], tmp["acceptance_rate"], 'o',
         markersize=2)  # Plot average class sizes against acceptances
plt.xlabel('average class size')
plt.ylabel('acceptance rate')

In [ ]:
# Add regression line to visualization:
y_hat = betas[0] * tmp["avg_class_size"] + \
    y_int  # slope-intercept form, y = mx + b
plt.plot(tmp["avg_class_size"], y_hat, color='orange', linewidth=0.5)

# add title, r-squared rounded to nearest thousandth
plt.title('R^2: {:.3f}, R: {:.3f}'.format(r_sqr, r))

From the Above plots, there's no evidence that the availability of material resources impacts object measures of achievement: the smaller the class and higher spending per pupil, the lower the achievement ratings and HSPHS acceptance rate, which is very unexpected. It is possible that there are some confounding factors we didn't rule out lead to this result. 
It is also possible that class size and per pupil spending from school does not matter too much because the availability of material resources for students is typically not determined by school, but  by their family. Perhaps the percentage of students living in households below the poverty line will better represent students' access to material resources in this case. 

In [ ]:
# Question 7
# proportion of schools accounts for 90% of all students accepted to HSPHS? 
# normalizing to get the percentage
sorted_acceptance = data[["acceptances",
                                 "school_name"]].sort_values(by="acceptances",
                                                             ascending=False)
normalized_sorted_accptance = sorted_acceptance["acceptances"] / \
    sorted_acceptance["acceptances"].sum()

plt.bar(sorted_acceptance["school_name"][:20], normalized_sorted_accptance[:20])
plt.xlabel('school')
plt.ylabel('% of total acceptance')

labels = sorted_acceptance["school_name"][:20]
plt.xticks(np.linspace(0, 19, 20), labels, rotation="vertical")

accumulated_percentage = normalized_sorted_accptance.cumsum()

sum(accumulated_percentage < 0.9) / len(accumulated_percentage)

From the ouput above, about 20.5% of schools account for 905 of students accepted to HSPHSs

In [ ]:
# Question 8
# Building the model
# Using regression model
# ethnicity identity is not included because one of these columns could be
# determined by other colunmns and it is not possible to reduce the dimension
# on these coulumns. They may break the validity of this prediction model.
glm_data = data[["per_pupil_spending",
                        "avg_class_size", "disability_percent",
                        "climate_rating", "poverty_percent", "ESL_percent",
                        "school_size", "climate_rating", "acceptance_rate",
                        "achievement_rating"]]
glm_data = glm_data.dropna()

# to tell which factor is most important, we need to z-score the data
zscored_glm_data = stats.zscore(glm_data)

X = zscored_glm_data[:, :-2]
Y = zscored_glm_data[:, -2]

regr = LinearRegression().fit(X, Y)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

plt.bar(np.linspace(0, 8, 8), betas)
plt.xlabel('variables')
plt.ylabel('betas')

labels = ["per_pupil_spending",
          "avg_class_size", "disability_percent",
          "climate_rating", "poverty_percent", "ESL_percent",
          "school_size", "climate_rating"]
plt.xticks(np.linspace(0, 8, 8), labels, rotation=45)
plt.title("Beta coefficients of each variable for prediting acceptance rate")

In [ ]:
Y = zscored_glm_data[:, -1]

regr = LinearRegression().fit(X, Y)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

plt.figure()
plt.bar(np.linspace(0, 8, 8), betas)
plt.xlabel('variables')
plt.ylabel('betas')

labels = ["per_pupil_spending",
          "avg_class_size", "disability_percent",
          "climate_rating", "poverty_percent", "ESL_percent",
          "school_size", "climate_rating"]
plt.xticks(np.linspace(0, 8, 8), labels, rotation=45)
plt.title(
    "Beta coefficients of each variable for predicting academic achievement")

 from the plots above, poverty rate is the determining factor on the HSPHS admission and achieving highs cores on objective measures of acceptance.

Whole summary on the report for question 9 and 10 and all other questions